In [43]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from sklearn.model_selection import GridSearchCV
np.random.seed(0)
from sklearn.linear_model import SGDClassifier
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.naive_bayes import BernoulliNB
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from lightgbm import LGBMClassifier
from xgboost import XGBClassifier
import sklearn

import category_encoders as ce


import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.optimizers import Adam, Nadam
from tensorflow.keras.layers import Input, Embedding, Reshape, GlobalAveragePooling1D
from tensorflow.keras.layers import Flatten, concatenate, Concatenate, Lambda, Dropout, SpatialDropout1D
from tensorflow.keras.layers import Reshape, MaxPooling1D,BatchNormalization, AveragePooling1D, Conv1D
from tensorflow.keras.layers import Activation, LeakyReLU
from tensorflow.keras.optimizers import SGD, Adam, Nadam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.regularizers import l2, l1_l2
from keras.losses import binary_crossentropy
import tensorflow_addons as tfa

In [19]:

def scaler(scaler, data, test=None):
    scaler.fit(data)  # Apply transform to both the training set and the test set.
    train_scale = scaler.transform(data)
    if test is not None:
        test_scale = scaler.fit_transform(test)

    return train_scale, test_scale, scaler





In [3]:
data = pd.read_table('data/dev.tsv')
test = pd.read_table('data/eval.tsv')

In [4]:
df = data.copy()

eval = test.copy()

In [5]:
from scipy import stats
def happy_sad(x):
    if x>df['valence'].mean():
        return 'happy'
    else:
        return 'sad'

def popularity_cat(x):
    if x>= 7:
        return 'high'
    elif x >= 4 and x < 7:
        return 'med'
    else:
        return 'low'

df['boringness'] = df['loudness'] + df['tempo'] + (df['energy']*100) + (df['danceability']*100)
df['valence_happy_sad'] = df['valence'].apply(lambda x: happy_sad(x))
df['loudness_plus_60'] = df['loudness'].apply(lambda x: x+60)
df['loudness_pos'] = df['loudness'].apply(lambda x: -1*x)
df['loudness_pos'] = np.sqrt(df['loudness_pos'])
df['boringness_plus_60'] = df['boringness'].apply(lambda x: x+60)
df['duration_ms_box_cox_trans'] = stats.boxcox(df['duration_ms'])[0]
df['acousticness_sqrt_trans'] = np.sqrt(df['acousticness'])
df['liveness_sqrt_trans'] = np.sqrt(df['liveness'])
df['popularity_sqrt_trans'] = np.sqrt(df['popularity'])
df['popularity_sqrt_trans_cat'] = df['popularity_sqrt_trans'].apply(lambda x: popularity_cat(x))
df['speechiness_sqrt_trans'] = np.sqrt(df['speechiness'])


df = df.fillna(value=0)
# df.describe().T

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [83]:


col = [
    'valence',
        # 'year',
        # 'acousticness',
        # 'artists',
         'danceability',
       'duration_ms',
        'energy',
        'explicit',
        # 'id',
        'instrumentalness',
        'key',
       'liveness',
        # 'loudness',
        # 'popularity',
        # 'speechiness',
        'tempo',
       #  'mode',
       # 'loudness_plus_60',
        'loudness_pos',
         # 'boringness',
       #  'valence_happy_sad',
       # 'boringness_plus_60',
       #  'duration_ms_box_cox_trans',
       'acousticness_sqrt_trans',
       #  'liveness_sqrt_trans',
       # 'popularity_sqrt_trans',
        'speechiness_sqrt_trans',
          # 'duration_ms_box_cox_trans_per_class',
    # 'popularity_sqrt_trans_cat'
        ]

X = df[col]
y = df['mode']

from imblearn.under_sampling import RandomUnderSampler,NearMiss,TomekLinks,ClusterCentroids
nm = NearMiss()
X_spl, y_spl = nm.fit_resample(X, y)

X_train, X_test, y_train, y_test = train_test_split(X_spl, y_spl,
                                                    test_size=0.3,
                                                    random_state=0,
                                                    shuffle=True)



In [84]:
from sklearn.preprocessing import *
X_train_scal, X_test_scal, x_scaler = scaler(MinMaxScaler(), X_train, X_test)

In [86]:
# Creating the model
model = Sequential()

# Inputing the first layer with input dimensions
model.add(Dense(128,
                activation='relu',
                input_dim=X_train.shape[1],
                kernel_initializer='uniform'))

model.add(Dropout(0.50))


model.add(Dense(64,
                kernel_initializer='uniform',
                activation='relu'))


model.add(Dropout(0.50))
model.add(Dense(32,
                kernel_initializer='uniform',
                activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(16,
                kernel_initializer='uniform',
                activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(8,
                kernel_initializer='uniform',
                activation='relu'))
model.add(Dropout(0.50))
model.add(Dense(1,
                kernel_initializer='uniform',
                activation='sigmoid'))

model.summary()

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_24 (Dense)             (None, 128)               1664      
_________________________________________________________________
dropout_18 (Dropout)         (None, 128)               0         
_________________________________________________________________
dense_25 (Dense)             (None, 64)                8256      
_________________________________________________________________
dropout_19 (Dropout)         (None, 64)                0         
_________________________________________________________________
dense_26 (Dense)             (None, 32)                2080      
_________________________________________________________________
dropout_20 (Dropout)         (None, 32)                0         
_________________________________________________________________
dense_27 (Dense)             (None, 16)               

In [68]:
def evaluate(y_true, y_pred):
    return f1_score(y_true, y_pred, average="macro")


In [87]:
#Creating an Stochastic Gradient Descent
sgd = SGD(lr = 0.01, momentum = 0.9)

# Compiling our model
model.compile(optimizer = 'Adam',
                   loss = 'binary_crossentropy',
                   metrics = [tfa.metrics.F1Score(num_classes=1, average='macro')])
#optimizers list
#optimizers['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',
    verbose=1,
    patience=10,
    mode='max',
    restore_best_weights=True)


# Fitting the ANN to the Training set
model.fit(X_train_scal, y_train, validation_split=0.3,callbacks=[early_stopping],
               batch_size = 32,
               epochs = 30, verbose=2)
scores = model.evaluate(X_test_scal, y_test, batch_size=32)
print("%s: %.2f%%" % (model.metrics_names[1], scores[1]*100))


Epoch 1/30
1226/1226 - 2s - loss: 0.6844 - f1_score: 0.6642 - val_loss: 0.6670 - val_f1_score: 0.6627
Epoch 2/30
1226/1226 - 2s - loss: 0.6731 - f1_score: 0.6642 - val_loss: 0.6578 - val_f1_score: 0.6627
Epoch 3/30
1226/1226 - 2s - loss: 0.6684 - f1_score: 0.6642 - val_loss: 0.6495 - val_f1_score: 0.6627
Epoch 4/30
1226/1226 - 2s - loss: 0.6577 - f1_score: 0.6643 - val_loss: 0.6469 - val_f1_score: 0.6627
Epoch 5/30
1226/1226 - 2s - loss: 0.6497 - f1_score: 0.6644 - val_loss: 0.6283 - val_f1_score: 0.6628
Epoch 6/30
1226/1226 - 2s - loss: 0.6477 - f1_score: 0.6646 - val_loss: 0.6300 - val_f1_score: 0.6631
Epoch 7/30
1226/1226 - 2s - loss: 0.6429 - f1_score: 0.6648 - val_loss: 0.6224 - val_f1_score: 0.6631
Epoch 8/30
1226/1226 - 2s - loss: 0.6404 - f1_score: 0.6648 - val_loss: 0.6181 - val_f1_score: 0.6632
Epoch 9/30
1226/1226 - 2s - loss: 0.6414 - f1_score: 0.6649 - val_loss: 0.6207 - val_f1_score: 0.6634
Epoch 10/30
1226/1226 - 2s - loss: 0.6400 - f1_score: 0.6652 - val_loss: 0.6215 - 

In [79]:
print('evaluation...')
y_p = model.predict_classes(X_test)
y_p =np.squeeze(y_p, axis=1)
score = evaluate(y_test, y_p)
print(f'score is {score}')

evaluation...
score is 0.3299377633892439


In [26]:
def WriteOnFile(name, y_eval):
    f = open(name, "w")
    f.write("Id,Predicted\n")
    for index, i in enumerate(y_eval):
        f.write(f"{index},{i}\n")
    f.close

eval['boringness'] = eval['loudness'] + eval['tempo'] + (eval['energy']*100) + (eval['danceability']*100)
eval['valence_happy_sad'] = eval['valence'].apply(lambda x: happy_sad(x))
eval['loudness_plus_60'] = eval['loudness'].apply(lambda x: x+60)
eval['loudness_pos'] = eval['loudness'].apply(lambda x: -1*x)
eval['loudness_pos'] = np.sqrt(eval['loudness_pos'])
eval['boringness_plus_60'] = eval['boringness'].apply(lambda x: x+60)
eval['duration_ms_box_cox_trans'] = stats.boxcox(eval['duration_ms'])[0]
eval['acousticness_sqrt_trans'] = np.sqrt(eval['acousticness'])
eval['liveness_sqrt_trans'] = np.sqrt(eval['liveness'])
eval['popularity_sqrt_trans'] = np.sqrt(eval['popularity'])
eval['speechiness_sqrt_trans'] = np.sqrt(eval['speechiness'])


eval = eval.fillna(value=0)

/home/omid/anaconda3/envs/ml/lib/python3.8/site-packages/pandas/core/series.py:726: RuntimeWarning: invalid value encountered in sqrt
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [135]:
test = eval[col]
# test = encoder.transform(test)
test_scal = x_scaler.transform(test)
test_svd = tsvd.transform(test_scal)
y_pred = clf.predict(test_svd)
WriteOnFile('submission.csv',y_pred)

In [108]:
test


,valence,year,danceability,duration_ms,energy,explicit,instrumentalness,key,liveness,tempo,loudness_pos,acousticness_sqrt_trans,speechiness_sqrt_trans
0,0.741,1962,0.465,147173,0.389,0,0.506000,11,0.1100,125.010,3.843176,0.997497,0.198746
1,0.928,1938,0.578,165133,0.234,0,0.000391,0,0.0784,172.403,3.507848,0.958123,0.319374
2,0.733,1998,0.681,333960,0.726,0,0.000019,9,0.1190,147.925,2.901896,0.356371,0.189209
3,0.914,2001,0.724,224427,0.731,0,0.003430,8,0.1770,101.602,2.675631,0.391152,0.180278
4,0.228,2015,0.594,172704,0.850,1,0.000000,2,0.1100,100.002,2.554408,0.170587,0.308869
...,...,...,...,...,...,...,...,...,...,...,...,...,...
34126,0.783,1965,0.770,155427,0.646,0,0.000000,9,0.1370,118.771,3.274599,0.754983,0.261534
34127,0.425,1975,0.373,320387,0.423,0,0.527000,0,0.1410,120.941,3.546407,0.502991,0.189209
34128,0.142,2000,0.156,315267,0.279,0,0.007810,0,0.1170,72.555,3.179308,0.979285,0.200000
34129,0.844,1982,0.438,225000,0.418,0,0.000000,2,0.3300,98.173,3.543445,0.146287,0.198494


In [ ]:
# cols = [
#     'valence',
#         'year',
#         # 'acousticness',
#         # 'artists',
#          'danceability',
#        # 'duration_ms',
#         'energy',
#         'explicit',
#         # 'id',
#         'instrumentalness',
#         'key',
#        'liveness',
#         # 'loudness',
#         # 'popularity',
#         # 'speechiness',
#         'tempo',
#        #  'mode',
#        # 'loudness_plus_60',
#         'loudness_pos',
#          # 'boringness',
#        #  'valence_happy_sad',
#        # 'boringness_plus_60',
#         'duration_ms_box_cox_trans',
#        'acousticness_sqrt_trans',
#        #  'liveness_sqrt_trans',
#        # 'popularity_sqrt_trans',
#         'speechiness_sqrt_trans',
#       # 'duration_ms_box_cox_trans_per_class'
#         ]
n/metrics/_classification.py:1043 f1_score  *